### Testando instalação de libs

In [ ]:
%pip install minio

### Import das libs

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from minio import Minio
from minio.error import S3Error

### Testando PySpark e driver do Postgresql

In [ ]:
spark = SparkSession.builder \
    .appName("CSV to PostgreSQL") \
    .config("spark.driver.extraClassPath", "/home/jovyan/notebooks/jars/postgresql-42.6.0.jar") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

csv_file_path = "/home/jovyan/notebooks/example.csv"

csv_schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),    
])

df = spark.read.csv(csv_file_path, header=True, schema=csv_schema)

postgres_url = "jdbc:postgresql://postgres:5432/prova"
postgres_properties = {
    "user": "postgres",
    "password": "password",
    "driver": "org.postgresql.Driver"
}
postgres_table = "users"

df.write \
    .format("jdbc") \
    .option("url", postgres_url) \
    .option("dbtable", postgres_table) \
    .option("user", postgres_properties["user"]) \
    .option("password", postgres_properties["password"]) \
    .option("driver", postgres_properties["driver"]) \
    .mode("overwrite") \
    .save()

print('Dados inseridos no banco de dados!')

df.show()

spark.stop()

### Testando a conexão com Minio

In [ ]:
minio_endpoint = 'minio:9000'
minio_access_key = 'minioaccesskey'
minio_secret_key = 'miniosecretkey'
bucket_name = 'raw'
local_file_path = '/home/jovyan/notebooks/example.csv'
minio_object_name = 'example.csv'

minio_client = Minio(minio_endpoint, access_key=minio_access_key, secret_key=minio_secret_key, secure=False)

if not minio_client.bucket_exists(bucket_name):
    minio_client.make_bucket(bucket_name)

try:
    minio_client.fput_object(bucket_name, minio_object_name, local_file_path)
    print(f"Arquivo '{minio_object_name}' enviado com sucesso para o MinIO.")
except S3Error as e:
    print(f"Erro ao enviar o arquivo para o MinIO: {e}")